# APS 4 - PROJETO DE SOFTWARE PARA ANÁLISE DE TRELIÇAS PLANAS

In [12]:
from funcoesTermosol import geraSaida, importa, plota
import numpy as np
from math import sqrt 

# Variáveis globais
matriz_um = np.array([[1, -1], [-1, 1]])

In [13]:
def gauss_seidel(A, b, tolerance, x):
    #x is the initial condition
    iter1 = 0
  
    while(True):
        iter1 = iter1 + 1
     
        x_old  = x.copy()
        
    
        for i in range(A.shape[0]):
            x[i] = (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,(i+1):], x_old[(i+1):])) / A[i ,i]
            
        
        LnormInf = max(abs((x - x_old)))/max(abs(x_old))   
        if  LnormInf < tolerance:
            print ("A solução do sistema na iteração", iter1, "é:", x)
            break
    
           
    return x

In [14]:
# numero de nos [nn]
# matriz dos nos [N]
# numero de membros [nm]
# matriz de incidencia [Inc]
# numero de cargas [nc]
# vetor carregamento [F]
# numero de restricoes [nr]
# vetor de restricoes [R]


class Node:
    def __init__(self, x, y, gl):
        self.x = x
        self.y = y
        self.gl = gl

        self.rx = None
        self.ry = None


class Elemento:
    def __init__(self, no1, no2, E, A):
        self.E = E
        self.A = A
        self.l = sqrt((no1.x - no2.x)**2 + (no1.y - no2.y)**2)
        self.cos = (no2.x - no1.x)/self.l
        self.sin = (no2.y - no1.y)/self.l
        self.T = np.array([[self.cos, self.sin, 0, 0],
                          [0, 0, self.cos, self.sin]])
        self.T_transposta = np.transpose(self.T)

        self.ke = np.dot(np.dot(self.T_transposta *
                         (self.E*self.A/self.l), matriz_um), self.T)

        self.gls = np.concatenate((no1.gl, no2.gl))


[nn, N, nm, Inc, nc, F, nr, R] = importa('entrada.xls')

sigma_tracao = 1570e6
sigma_compressao = sigma_tracao

gdl_global = []
# Criar um objeto Node para cada nó
NODES = []
for n in range(nn):
    gdl = [((n+1)*2)-1, (n+1)*2]
    gdl_global = np.concatenate((gdl_global, gdl))
    new_node = Node(N[0, n], N[1, n], gdl)
    NODES.append(new_node)

# Node1 tem restrição em x
# Node2 tem restrição em x e y

# Criar um objeto Elemento para cada membro
ELEMENTOS = []
for m in range(nm):
    new_element = Elemento(NODES[int(Inc[m, 0])-1], NODES[int(Inc[m, 1])-1], float(Inc[m, 2]), float(Inc[m, 3]))
    ELEMENTOS.append(new_element)

# Cria matriz de rigidez para superposição das matrizes de rigidez dos elementos
K = np.zeros((2*nn, 2*nn))

AUXS = [e.gls for e in ELEMENTOS]
for aux, elemento in zip(AUXS,ELEMENTOS):
    for i in range(len(aux)):
        for j in range(len(aux)):
            K[aux[i]-1, aux[j]-1] += elemento.ke[i, j]

# Vetor global de forças concentradas (Aula 25 -> Slide 22)
PG = F

# Aplicando condição de contorno (Aula 25 -> Slide 23)
# Removendo linhas e colunas que correspondem aos graus de liberdade das reações dos nós
# Documentação para o np.delete: https://note.nkmk.me/en/python-numpy-delete/#:~:text=Using%20the%20NumPy%20function%20np,from%20the%20NumPy%20array%20ndarray%20.&text=Specify%20the%20axis%20(dimension)%20and,a%20slice%20or%20a%20list.
R = R.astype(int).flatten()
print(R)
K_CC = np.delete(np.delete(K, R, 0), R, 1)
PG_CC = np.delete(PG, R, 0)

# Calcular os deslocamentos nodais (Aula 25 -> Slide 24)

deslocamentos = gauss_seidel(K_CC, PG_CC, 1e-10, np.zeros(len(PG_CC)))

# Calcular reações de apoio (Aula 25 -> Slide 25)



# Calcular deformação e tensão em cada elemento (Aula 25 -> Slide 8)

# Calcular as forças internas -> Tensão do elemento = Força/Área


[0 2 3]
A solução do sistema na iteração 23 é: [-9.52380952e-07  1.60714286e-06 -4.01785714e-06]


C:\Users\renat\AppData\Local\Temp\ipykernel_5580\1552097990.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  LnormInf = max(abs((x - x_old)))/max(abs(x_old))
